In [28]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import Imputer
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import classification_report_imbalanced

In [29]:
#Reading data files
path = r'C:\Users\az9826\Desktop\Machine_learning'
filename_hr1 = path+'\\'+'HR_bill_status_contemporary.json'
filename_hr2 = path+'\\'+'HR_bill_status_modern.json'
f_hm = path+'\\'+'Majority Party Chairmanships - House.csv'
f_lc = path+'\\'+'legislators-current.csv'
f_lh = path+'\\'+'legislators-historical.csv'

In [30]:
#Getting the details about the congress and which party was in majority for that bill
df_hm = pd.read_csv(f_hm)
df_hm = df_hm[['Congress', 'MajParty']]
df_hm = df_hm.drop_duplicates()

In [31]:
#Data preprocessing : fetching sponsors name and to which party they belong from legislators historical table
#There was a seperate fiel for full name in the data but the data was ambegiuous, therefore create the full name using the 
#other two fields present that is first name and last name
#keeping only those records where party type is rep that is house of representatives 
#And keeping only those records where party is either Republican or Democrat sinced there were many other parties as well 
# but these two are of our interest 
df_lc = pd.read_csv(f_lc)
df_lh = pd.read_csv(f_lh)
df_lh['full_name'] = df_lh['last_name'].fillna('')+', '+df_lh['first_name'].fillna('')
df_lh = df_lh[df_lh['type'] == 'rep']
df_lh = df_lh[['full_name', 'party']]
party = ["Republican", "Democrat"]
df_lh = df_lh[df_lh['party'].isin(party)]

In [32]:
#Appending the two senate files because we have collected the data from two different sources
df_hr1 = pd.read_json(filename_hr1)
df_hr2 = pd.read_json(filename_hr2)
df_hr = df_hr1.append(df_hr2, ignore_index=True)
df_hr = df_hr.fillna(np.nan)


C:\Users\az9826\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [33]:
# normalizingand scaling the data by creating flags 
#if there is any data in the cosponsor field taht is any non missing value make the flag 1 else 0 
#if there is non missing value in the field amendments make the flag 1 else 0
#if there is non missing value in the field  popular_title make the flag 1 else 0
#if there is non missing value in the field  related_bills make the flag 1 else 0
# creating full name for the sponsor , keeping only first and last name
df_hr['cosponsor_flg'] = np.where(df_hr['cosponsors'].apply(len)==0, 0, 1)
df_hr['amendments_flg'] = np.where(df_hr['amendments'].apply(len)==0, 0, 1)
df_hr['popular_flg'] = np.where(df_hr['popular_title'].isnull(), 0, 1)
df_hr['related_bills_flg'] = np.where(df_hr['related_bills'].apply(len)==0, 0, 1)
df_hr['sponsor_nm'] = [d.get('name') for d in df_hr.sponsor]
df_hr['full_name'] = df_hr['sponsor_nm'].str.split(' ',expand = True)[0]+' '+df_hr['sponsor_nm'].str.split(' ',expand = True)[1] 
df_hr['Congress'] = pd.to_numeric(df_hr['congress'], errors='coerce')

In [36]:
#merging files on sponsor names and congress

df_hr1 = pd.merge(df_hr, df_lh, how='inner', on=['full_name'])
df_hr2 = pd.merge(df_hr1, df_hm, how='inner', on=['Congress'])

df_hr2 = df_hr2[df_hr2['party'].isin(party)]
df_hr2['party_flg'] = np.where(df_hr2['party'] == "Republican", 0, 1)
df_hr2['majority_flg'] = np.where(df_hr2['MajParty'] == "R", 0, 1)
#statuses = ["PASSED:BILL", "ENACTED:SIGNED"]
statuses = ["PASSED:BILL", "ENACTED:SIGNED", "REPORTED", "PROV_KILL:SUSPENSIONFAILED", "PROV_KILL:CLOTUREFAILED",
           "FAIL:ORIGINATING:HOUSE", "PASSED:SIMPLERES", "PASSED:CONSTAMEND", "PASS_OVER:HOUSE", "PASSED:CONCURRENTRES",
           "FAIL:SECOND:HOUSE", "PASS_BACK:HOUSE", "PROV_KILL:PINGPONGFAIL", "CONFERENCE:PASSED:HOUSE", "PROV_KILL:VETO",
           "VETOED:POCKET","VETOED:OVERRIDE_FAIL_ORIGINATING:SENATE","VETOED:OVERRIDE_FAIL_ORIGINATING:HOUSE", "VETOED:OVERRIDE_PASS_OVER:HOUSE",
           "ENACTED:VETO_OVERRIDE", "ENACTED:TENDAYRULE"]
df_hr2['Class'] = np.where(df_hr2['status'].isin(statuses), 1, 0)


In [37]:
#Final Feature selection
#our class field is Status
df_hr = df_hr2[['bill_id', 'cosponsor_flg','amendments_flg', 'popular_flg', 'related_bills_flg', 'party_flg', 'majority_flg', 'Class']]
df_hr = df_hr.reset_index(drop = True)


In [38]:
#taking 80% training datafeatures = list(df_hr.columns[1:7])
features = list(df_hr.columns[1:7])
traindatasize = int(len(df_hr)*.8)
testdatasize = len(df_hr) - traindatasize

In [39]:
#assigning data for tarining and test
traindata = df_hr[0:traindatasize]
testdata = df_hr[traindatasize:len(df_hr)+1]


df2 = traindata
y_train = df2["Class"]
X_train = df2[features]

df3 = testdata
y_test = df3["Class"]
X_test = df3[features]

In [40]:
#applying random forest 
rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
rf.fit(X_train, y_train)
y_predicted_rf = rf.predict(X_test)
y_score_rf = rf.score(X_test,y_test)

y_score_rf

0.8866472545757071

In [41]:
#applying decision tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_predicted = dt.predict(X_test)
y_score = dt.score(X_test,y_test)

y_score

0.8864392678868552

In [42]:
#applying SVM
d_svm = svm.SVC()
d_svm.fit(X_train, y_train)
y_predicted_svm = d_svm.predict(X_test)
y_score_svm = d_svm.score(X_test,y_test)

y_score_svm

0.8866472545757071